In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import cv2
import time
import glob
import os
import pandas
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tqdm import tqdm
import random

In [ ]:
class DataLoader(keras.utils.Sequence):
    def __init__(self,csv='/kaggle/input/rsnasubmissionresult/result.csv',height=256,width=256,inchannels=1,batch_size=16,shuffle=True,\
                modeltype='imagenet',maxlen_slices=50,return_feat=True):
        self.batch_size = batch_size
        self.csv = csv
        self.base_dir = '/kaggle/input/test-train-08thresh-mgmt-tumorimages/DATATUMORONLY/train/'
        self.width = width
        self.height = height
        self.return_feat = return_feat
        self.shuffle = shuffle
        self.inchannels = inchannels
        self.model_type = modeltype #imagenet or tumor
        self.modelarch = 'resnet50'
        self.maxlen_slices = maxlen_slices
        
        if self.inchannels == 1:
            self.imgreadmode = 'grayscale'
        elif self.inchannels == 1:
            self.imgreadmode = 'rgb'
        if self.return_feat:
            self.load_feature_extractor()
        self.load_data()
        self.on_epoch_end()
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.all_pat_ids)
            
    def load_data(self):#memory fill ho jaegi is se
        df = pd.read_csv(self.csv)
        dset = tqdm(df.iterrows(),total=len(df))
        self.use_cols = ['FLAIR'] #,'T2w']
        self.ignore_ids = [109, 123, 709]
        self.min_frames = 5
        self.data = {}
        self.class_map = {0:'Tumor',1:'MGMT'}

        for i,row in dset:
            #folder_id, FLAIR, T1w, T1wCE, T2w, BraTS21ID, MGMT_value = row
            pat_id = int(row['folder_id'])
            if pat_id in self.ignore_ids:
                print('Skipping PATID:',pat_id)
                continue
            target = row['MGMT_value']
            images = []
            pat_id = str(pat_id).zfill(5)
            for col in self.use_cols:
                images.extend(glob.glob(os.path.join(self.base_dir,pat_id,col.lower(),'*.png')))
            if len(images)>self.min_frames:
                self.data[pat_id] = {}
                self.data[pat_id]['images'] = images[:self.maxlen_slices]#issue here
                self.data[pat_id]['label'] = int(target)
        
        self.all_pat_ids = list(self.data.keys())
        df = None
        dset = None
        print('DATA:>> ',len(self.data),' | IDS',len(self.all_pat_ids))
        
    def load_feature_extractor(self):
        self.feat_batchsize = 16
        if self.model_type == 'imagenet':
            base_model = ResNet50(weights='imagenet')#input_shape
            x = layers.Reshape((-1,1))(base_model.layers[-2].output)
            x = layers.AveragePooling1D(pool_size=4)(x)
            out = layers.Reshape((-1,))(x)
            self.featext = Model(inputs=base_model.input, outputs=[out,base_model.layers[-1].output])
            self.outdim = 512
            base_model = None
            self.featext_preprocessor = preprocess_input
            self.featext_loadmode = 'rgb'
        else:
            path = '/kaggle/input/classifier4mods-finetune/FINAL_MODELALLMods_ep5_acc98_finetuned.h5'
            base_model = load_model(path)
            self.featext = Model(inputs=base_model.input, outputs=[base_model.layers[-2].output,base_model.layers[-1].output])
            self.outdim = base_model.layers[-2].output_shape[-1]
            base_model = None
            self.featext_preprocessor = lambda x:x/255.
            self.featext_loadmode = 'grayscale'
            
            
    def load_image_array(self,img_path,mode='rgb'):
        img = image.load_img(img_path,target_size=(self.height,self.width),color_mode=mode)
        img = image.img_to_array(img)
        return img
    
    def one_hot_encoder(self,y):
        b = np.zeros((len(y), 2))
        b[np.arange(len(y)),y] = 1
        return b
    
    def __len__(self):
        return int(len(self.data)/self.batch_size)
    
    def pad_depth_feat(self,featslist,shp):
        padded_feat = np.zeros(shp)
        mask = np.zeros((len(featslist), shp[1]))
        for i,feat in enumerate(featslist):
            padded_feat[i,:feat.shape[0]] = feat
            mask[i,:feat.shape[0]] = 1
        
        return padded_feat,mask
            
    
    def create_2d_mask(self,mask):
        b,t = mask.shape
        mask2d = np.zeros((b,t,t))
        for i in range(b):
            for j in range(t):
                mask2d[i][j] = mask[i]
        return mask2d
    
    def __getitem__(self,index):
        full_batch_feat = []
        full_batch_target = []
        depthlist = []
        for patid in self.all_pat_ids[index*self.batch_size:(index+1)*self.batch_size]:
            full_batch_target.append(self.data[patid]['label'])
            imagespath = self.data[patid]['images']
            
            if self.return_feat:
                
                allfeats = np.zeros((len(imagespath),self.outdim))
                for i in range(0,len(imagespath),self.feat_batchsize):
                    batchimgspath = imagespath[i:i+self.feat_batchsize]

                    batchimgs = [self.load_image_array(path,self.featext_loadmode) for path in batchimgspath]
                    batchimgs = [self.featext_preprocessor(img) for img in batchimgs]
                    batchimgs = np.array(batchimgs)

                    if len(batchimgs.shape)<4:
                        batchimgs = batchimgs[:,:,:,None]#ye kyu chahiye


                    batchfeat,_ = self.featext(batchimgs)
                    #print('patid',patid,'batchfeat',batchfeat.shape)
                    allfeats[i:i+self.feat_batchsize] = batchfeat

                    batchfeat = None
                    batchimgs = None
                    _ = None

                #print('patid',patid,'allfeats',allfeats.shape)
            else:
                allfeats = np.zeros((len(imagespath),self.height,self.width,self.inchannels))
                for i in range(0,len(imagespath)):
                    path = imagespath[i]
                    img = self.load_image_array(path,self.imgreadmode)
                    img /= 255.
                    if len(img.shape) == 2:
                        img = img[:,:,None]
                        
                    allfeats[i,:,:,:] = img
                    
            full_batch_feat.append(allfeats)
            depthlist.append(allfeats.shape[0])
        
       
        #print('full_batch_feat',len(full_batch_feat))
        max_len = max(depthlist)
        if self.return_feat:
            shp = (len(full_batch_feat), max_len, self.outdim)
        else:
            shp = (len(full_batch_feat), max_len, self.height,self.width,self.inchannels)
        full_batch_feat, mask = self.pad_depth_feat(full_batch_feat,shp)
        mask = self.create_2d_mask(mask)
        full_batch_target = self.one_hot_encoder(full_batch_target)
        #print(f'OutBatch: {full_batch_feat.shape} || Label: {full_batch_target.shape} || Mask: {mask.shape}')

        return [full_batch_feat,mask],full_batch_target,
        

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, mask, training):
        attn_output = self.att(inputs, inputs,attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

In [ ]:
IH,IW = 224,224
modeltype = 'imagenet'
maxlen_slices = 100
bs = 16
gen = DataLoader(modeltype=modeltype,height=IH,width=IW,batch_size=bs,maxlen_slices=maxlen_slices,inchannels=1,return_feat=True)
if 1:
    for i,sample in enumerate(gen):
        [full_batch_feat,mask],full_batch_target = sample
        print(full_batch_feat.shape,full_batch_target.shape,mask.shape)
        break

In [ ]:
layers.Reshape

In [ ]:
embed_dim = 512  # Embedding size for each token
num_heads = 8  # Number of attention heads
ff_dim = embed_dim*4  # Hidden layer size in feed forward network inside transformer


inputs = layers.Input(shape=(None,embed_dim))
input_mask = layers.Input(shape=(None,None))
#pos_layer = PositionEmbedding(maxlen_slices, embed_dim)
#x = pos_layer(inputs)
transformer_block_1 = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block_1(inputs,input_mask)
transformer_block_2 = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block_2(x,input_mask)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = keras.Model(inputs=[inputs,input_mask], outputs=outputs)
model.summary()

In [ ]:
embed_dim = 512  # Embedding size for each token
num_heads = 8  # Number of attention heads
ff_dim = embed_dim*4  # Hidden layer size in feed forward network inside transformer


inputs = layers.Input(shape=(None,embed_dim))
input_mask = layers.Input(shape=(None,None))
path = '/kaggle/input/classifier4mods-finetune/FINAL_MODELALLMods_ep5_acc98_finetuned.h5'
base_model = load_model(path)
featext = Model(inputs=base_model.input, outputs=[base_model.layers[-2].output,base_model.layers[-1].output])
#pos_layer = PositionEmbedding(maxlen_slices, embed_dim)
#x = pos_layer(inputs)
transformer_block_1 = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block_1(inputs,input_mask)
transformer_block_2 = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block_2(x,input_mask)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = keras.Model(inputs=[inputs,input_mask], outputs=outputs)
model.summary()

In [ ]:
print(full_batch_feat.shape,full_batch_target.shape,mask.shape)
out = model([full_batch_feat,mask])
print(out.shape)

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True,metrics="accuracy",mode=max
)

In [ ]:
model.fit_generator(
    gen,
    steps_per_epoch=len(gen),
    epochs=50,
    verbose=1,
    callbacks=[checkpoint_cb],
)
model.save('FULL_TRANSFORMER_MODEL.h5')